In [1]:
#Importing usefull libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import math
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
#Paths
data_path="D://ML/Kaggle/ELL/data/"
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [3]:
df_modeling = pd.read_csv(data_path+"train.csv")
df_oot = pd.read_csv(data_path+"test.csv")
df_sample = pd.read_csv(data_path+"sample_submission.csv")

In [4]:
print(df_modeling.shape)
df_modeling.head(2)

(3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [5]:
df_train, df_test = train_test_split(df_modeling,test_size=0.20, random_state=123)

In [6]:
#Useful functions
def inputfromDF(df):
    X = df['full_text']
    Y1 = df['cohesion']
    Y2 = df['syntax']
    Y3 = df['vocabulary']
    Y4 = df['phraseology']
    Y5 = df['grammar']
    Y6 = df['conventions']
    return X, Y1, Y2, Y3, Y4, Y5, Y6

def roundoffScore(n):
    x1=math.floor(n)
    if n>x1+0.75:
        return x1+1.0
    elif n>x1+0.25:
        return x1+0.5
    else:
        return x1

def MCRMSE(y_test, y_pred):
    mse_list=[]
    target_names = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
    for i in range(y_test.shape[1]):
        mse_list.append(mean_squared_error(y_test[:,i], y_pred[:,i]))
        print("MSE for {} : {:.4f}".format(target_names[i], mean_squared_error(y_test[:,i], y_pred[:,i])))
    return sum(mse_list)/y_test.shape[1]

In [7]:
X_train, Y1_train, Y2_train, Y3_train, Y4_train, Y5_train, Y6_train = inputfromDF(df_train)
X_test, Y1_test, Y2_test, Y3_test, Y4_test, Y5_test, Y6_test = inputfromDF(df_test)

### Functional regression model with multiple outputs for each target

In [8]:
t1=datetime.datetime.now()

In [9]:
bert_preprocess = hub.KerasLayer(preprocessor_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [10]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
reduceLR=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1,patience=3,mode="auto",min_delta=0.00001,min_lr=0.00001)
es=tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.00001,patience=5,mode="auto",restore_best_weights=True)

In [11]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
input_layer = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
x1 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output1 = tf.keras.layers.Dense(units='1')(x1)
x2 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output2 = tf.keras.layers.Dense(units='1')(x2)
x3 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output3 = tf.keras.layers.Dense(units='1')(x3)
x4 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output4 = tf.keras.layers.Dense(units='1')(x4)
x5 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output5 = tf.keras.layers.Dense(units='1')(x5)
x6 = tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer='he_normal')(input_layer)
output6 = tf.keras.layers.Dense(units='1')(x6)
model = tf.keras.Model(inputs=[text_input], outputs = [output1,output2,output3,output4,output5,output6])

In [12]:
model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mean_squared_error'])

In [15]:
model.fit(X_train, [Y1_train, Y2_train, Y3_train, Y4_train, Y5_train, Y6_train], validation_data=(X_test, [Y1_test, Y2_test, Y3_test, Y4_test, Y5_test, Y6_test]), epochs=100, batch_size=64, verbose=1,callbacks=[es,reduceLR])

Epoch 1/100

49/49 [==============================] - 57s 931ms/step - loss: 51.9688 - dense_1_loss: 2.9737 - dense_3_loss: 11.5139 - dense_5_loss: 6.2814 - dense_7_loss: 11.6492 - dense_9_loss: 8.9435 - dense_11_loss: 10.6072 - dense_1_mean_squared_error: 2.9737 - dense_3_mean_squared_error: 11.5139 - dense_5_mean_squared_error: 6.2814 - dense_7_mean_squared_error: 11.6492 - dense_9_mean_squared_error: 8.9435 - dense_11_mean_squared_error: 10.6072 - val_loss: 2.2833 - val_dense_1_loss: 0.3727 - val_dense_3_loss: 0.4202 - val_dense_5_loss: 0.3483 - val_dense_7_loss: 0.3878 - val_dense_9_loss: 0.3957 - val_dense_11_loss: 0.3586 - val_dense_1_mean_squared_error: 0.3727 - val_dense_3_mean_squared_error: 0.4202 - val_dense_5_mean_squared_error: 0.3483 - val_dense_7_mean_squared_error: 0.3878 - val_dense_9_mean_squared_error: 0.3957 - val_dense_11_mean_squared_error: 0.3586 - lr: 0.0100

Epoch 2/100

49/49 [==============================] - 44s 893ms/step - loss: 2.2699 - dense_1_loss: 0.40

In [16]:
model.save(data_path+"models/bert_model_1.h5")
print("Total time taken: ", datetime.datetime.now()-t1)

Total time taken:  0:19:49.646956


### Loading model

In [13]:
model = tf.keras.models.load_model(data_path+"models/bert_model_1.h5",custom_objects={'KerasLayer':hub.KerasLayer})

In [17]:
Y_pred = model.predict(X_test)

25/25 [==============================] - 10s 315ms/step


In [18]:
Y_test = np.vstack([Y1_test, Y2_test, Y3_test, Y4_test, Y5_test, Y6_test]).T
Y_pred1 = np.vstack([roundoffScore(j) for j in i] for i in np.hstack([x for x in Y_pred]))

C:\Users\Prashant\AppData\Local\Temp\ipykernel_9528\1000018708.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

  Y_pred1 = np.vstack([roundoffScore(j) for j in i] for i in np.hstack([x for x in Y_pred]))


In [20]:
MCRMSE(Y_test, Y_pred1)

MSE for cohesion : 0.3209

MSE for syntax : 0.2851

MSE for vocabulary : 0.2471

MSE for phraseology : 0.3004

MSE for grammar : 0.3193

MSE for conventions : 0.2963


0.29485951468710087

### Model Prediction on OOT data

In [23]:
X_oot = df_oot['full_text']

In [24]:
Y_pred_oot = model.predict(X_oot)

1/1 [==============================] - 0s 28ms/step


In [26]:
Y_pred_oot1 = np.vstack([roundoffScore(j) for j in i] for i in np.hstack([x for x in Y_pred_oot]))

C:\Users\Prashant\AppData\Local\Temp\ipykernel_9528\604317238.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

  Y_pred_oot1 = np.vstack([roundoffScore(j) for j in i] for i in np.hstack([x for x in Y_pred_oot]))


In [32]:
df_oot['cohesion'] = Y_pred_oot1[:,0]
df_oot['syntax'] = Y_pred_oot1[:,1]
df_oot['vocabulary']= Y_pred_oot1[:,2]
df_oot['phraseology']= Y_pred_oot1[:,3]
df_oot['grammar']= Y_pred_oot1[:,4]
df_oot['conventions']= Y_pred_oot1[:,5]

In [33]:
df_oot

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,when a person has no experience on a job their...,3.0,2.5,3.0,3.0,2.5,2.5
1,000BAD50D026,Do you think students would benefit from being...,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",3.5,3.0,3.5,3.5,3.0,3.5


In [37]:
df_oot.drop(columns=['full_text']).to_csv(data_path+"submission1.csv",index=False)